In [1]:
from sklearn.cluster import KMeans
import tensorflow as tf
import matplotlib.pyplot as plt
import argparse
import utils
import cv2
import os
import shutil
from PIL import Image

#folder='../../my_data/UCF-Brox/flow/jpg/'
#outputfolder='../../my_data/UCF-Brox/flow/key_jpg/'
folder='../../my_data/UCF-Brox/flow/rgb/jpg/'
outputfolder='../../my_data/UCF-Brox/flow/test-Key-1/jpg'
import time



In [2]:
def histImage(image):
    # reshape the image to be a list of pixels
    image = image.reshape((image.shape[0] * image.shape[1], 3))

    # cluster the pixel intensities
    clt = KMeans(n_clusters = 7, max_iter=300)
    clt.fit(image)
    
    # build a histogram of clusters and then create a figure
    # representing the number of pixels labeled to each color
    hist = utils.centroid_histogram(clt)
    #print("hist", hist)
    #print("clt.cluster_centers_",clt.cluster_centers_)
    bar = utils.plot_colors(hist, clt.cluster_centers_)
    
    return bar 
    

In [3]:
def compareHSV(bar0, bar1, a_margin=0, nc_margin=0, m_margin=0):
    newColor=[]
    for b in bar1:
        
        flag=False
        for bb in bar0:
            if (abs(b[1]-bb[1])<=a_margin) and (abs(b[2]-bb[2])<=m_margin):
                flag=True       
                
        if flag==False:#new color not in range of any exiting color
            if b[0] > nc_margin  or b[2] > m_margin:
                newColor.append(b)
            
    
    for b in bar0:
        flag1=False
        for bb in bar1:
            if (abs(b[1]-bb[1])<=a_margin) and (abs(b[2]-bb[2])<=m_margin):
                flag1=True       
                
        if flag1==False:#new color not in range of any exiting color
            if b[0] > nc_margin or b[2] > m_margin:
                newColor.append(b)
    return newColor
            
                

In [4]:
def excludeBK(bar):
    foregroundBar=[]
    
    for b in bar:
        foregroundBar.append((b[0],b[1],b[2]*b[3]))
    #return foregroundBar
    sorted_list = sorted(foregroundBar,key=lambda t: t[2])
    
    if sorted_list[0][0]>0.5:
        sorted_list.pop(0)
    #print(sorted_list)
    return sorted_list

In [5]:
def createDirectories(processed_vid):           
    if not os.path.exists(processed_vid):
        os.makedirs(processed_vid)
    else:
        files = os.listdir(processed_vid)
        for f in files:
            if os.path.isfile(os.path.join(processed_vid,f)):
                os.remove(os.path.join(processed_vid,f))

for action in os.listdir(folder):   
    actiobclass=os.path.join(folder,action)
    #print(actiobclass)
    for vid in os.listdir(actiobclass):
        if vid.endswith('.avi'):
            vidName=os.path.join(actiobclass,vid)
            #print(vid)
            outputPath=os.path.join(outputfolder,action, vid)
            #print(outputPath)
            createDirectories(outputPath)

In [6]:

for action in os.listdir(folder):
    
    actiobclass=os.path.join(folder,action)
    print(actiobclass)
    for vid in os.listdir(actiobclass):
        # get the start time
        st = time.time()
        if vid.endswith('.avi'):
            vidName=os.path.join(actiobclass,vid)
            print(vidName)
            outputPath=os.path.join(outputfolder,action, vid)
            createDirectories(outputPath)
            flows_path=[]
            keyframes=[]
            for opticalFow in os.listdir(vidName):
                if opticalFow.endswith(".jpg"):
                    opticalFlow=os.path.join(vidName,opticalFow)
                    flows_path.append(opticalFlow)
            if len(flows_path) > 0:
                flows_path.sort(key = lambda x: int(x.split('.')[-2].split('_')[-1]))
                #for x in flows_path:
                #    print(x)
                image1=cv2.imread(flows_path[0])
                image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
                #down Sample
                '''plt.figure()
                plt.axis("off")
                plt.imshow(image1)'''

                image1 = cv2.pyrDown(image1)

                keyframes.append(flows_path[0])
                for image in range(1, len(flows_path)):
                    print(flows_path[image])
                    image=flows_path[image]
                    #print(image1)

                    bar0=histImage(image1)

                    print(bar0)
                    # show our color bart
                    '''plt.figure()
                    plt.axis("off")
                    plt.imshow(bar0)
                    plt.show()
                    break'''
                    image2 = cv2.imread(image) 
                    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB) 
                    #down Sample
                    image2= cv2.pyrDown(image2)

                    bar1=histImage(image2)

                    bar0=excludeBK(bar0) 
                    bar1=excludeBK(bar1)

                    if len(compareHSV(bar0, bar1,15,0.2,0.2)) > 1:
                        keyframes.append(image)
                        image1=image2

                #print(len(keyframes))
                for i in keyframes:
                    #print(i)
                    shutil.copy(i, os.path.join(outputPath,i.split(os.path.sep)[-1]))
                    #print(os.path.join(outputPath,i.split(os.path.sep)[-1]))
                # get the end time
                et = time.time()

                # get the execution time
                elapsed_time = et - st
                print('Execution time:', elapsed_time, 'seconds')
            
   


../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/.avi
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_2.jpg
[(0.020729166666666667, 250, 0.38658414018887116, 0.9901763720563603), (0.84984375, 281, 0.006980687669530636, 0.9909972397367343), (0.04625, 295, 0.16230812961910385, 0.9643256626213057), (0.01328125, 352, 0.5144346743444109, 0.9881584006151478), (0.02802083333333333, 353, 0.2709294369523488, 0.9907500546687075), (0.03557291666666667, 277, 0.34182672330208985, 0.9934181859878456), (0.006302083333333333, 52, 0.3369285907492568, 0.9585480473181005)]
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_3.jpg
[(0.14442708333333334, 350, 0.053687213624967596, 0.997839106793096), (0.035104166666666665, 342, 0.35638429532891147, 0.9872694478384826), (0.6816145833333334, 335, 0.

../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_17.jpg
[(0.6415625, 342, 0.01255230392367007, 0.998378906685259), (0.010572916666666666, 308, 0.8421779290228488, 0.993431855500821), (0.07375, 345, 0.19234815240411116, 0.9977816550348954), (0.057604166666666665, 345, 0.13566371366563396, 0.9965961067971493), (0.0060416666666666665, 314, 0.5701700260523797, 0.9862068965517241), (0.19432291666666668, 344, 0.04804592137385258, 0.9988595813559946), (0.016145833333333335, 313, 0.3115636049144045, 0.9946363061353574)]
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_18.jpg
[(0.016145833333333335, 313, 0.3115636049144045, 0.9946363061353574), (0.6415625, 342, 0.01255230392367007, 0.998378906685259), (0.0060416666666666665, 314, 0.5701700260523797, 0.9862068965517241), (0.05765625, 345, 0.13566612043807202, 0.9965920966399207), (0.010572916666666666, 308, 0.8421779290228488, 0.993431855500821), (0.194

../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_32.jpg
[(0.31046875, 1, 0.0032400571611924615, 0.9971441822828778), (0.024322916666666666, 0, 0.6174584533756894, 0.9939203090229668), (0.04895833333333333, 346, 0.2194547909586111, 0.9968994186409951), (0.09666666666666666, 348, 0.12163823541923664, 0.9939009566090586), (0.029427083333333333, 346, 0.3788157949834616, 0.9945236855804268), (0.42380208333333336, 340, 0.04405573690450804, 0.99894117165116), (0.06635416666666667, 350, 0.5408210560142266, 0.9961892449287406)]
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_33.jpg
[(0.7093229166666667, 1, 0.014698506745995533, 0.9978326127425784), (0.05286458333333333, 350, 0.55739065353877, 0.9963218390804597), (0.09958333333333333, 1, 0.21854179308642205, 0.9981418745508674), (0.02984375, 1, 0.6415939555751307, 0.9945864558737979), (0.030052083333333333, 355, 0.35911280829979986, 0.9948529411764706

[(0.7277604166666667, 3, 0.020072908067182954, 0.9982465739587192), (0.04864583333333333, 3, 0.7174310920684096, 0.9956249737582399), (0.02515625, 13, 0.4262791401755985, 0.9963800904977375), (0.04161458333333334, 11, 0.22354382184968308, 0.9977619082676875), (0.045260416666666664, 0, 0.5341404204824332, 0.9963852975021998), (0.056875, 8, 0.095667798538769, 0.9982243309789544), (0.0546875, 5, 0.8662685940970782, 0.9970233426704014)]
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_48.jpg
[(0.7277604166666667, 3, 0.020072908067182954, 0.9982465739587192), (0.0546875, 5, 0.8662685940970782, 0.9970233426704014), (0.045260416666666664, 0, 0.5341404204824332, 0.9963852975021998), (0.056875, 8, 0.095667798538769, 0.9982243309789544), (0.04864583333333333, 3, 0.7174310920684096, 0.9956249737582399), (0.02515625, 13, 0.4262791401755985, 0.9963800904977375), (0.04161458333333334, 11, 0.22354382184968308, 0.9977619082676875)]
../../my_data/UCF-Brox/

../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_62.jpg
[(0.7681770833333333, 203, 0.00540541405334578, 0.9971600600373041), (0.04416666666666667, 191, 0.5477321433541513, 0.9969385867554569), (0.028854166666666667, 199, 0.873289619662624, 0.9953422524244354), (0.024010416666666666, 202, 0.27210895993566525, 0.9943175534856026), (0.013333333333333334, 227, 0.45015482737902646, 0.9943474264705883), (0.06916666666666667, 191, 0.7319824091415432, 0.9964800377982518), (0.05229166666666667, 199, 0.10610858132626876, 0.9967268182173268)]
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_63.jpg
[(0.7681770833333333, 203, 0.00540541405334578, 0.9971600600373041), (0.06916666666666667, 191, 0.7319824091415432, 0.9964800377982518), (0.05229166666666667, 199, 0.106163579135282, 0.9967261730562872), (0.024010416666666666, 202, 0.27235002100606265, 0.9943137254901961), (0.028854166666666667, 199, 0.87328961

[(0.7681770833333333, 203, 0.00540541405334578, 0.9971600600373041), (0.028854166666666667, 199, 0.873289619662624, 0.9953422524244354), (0.04416666666666667, 191, 0.5477321433541513, 0.9969385867554569), (0.06916666666666667, 191, 0.7319824091415432, 0.9964800377982518), (0.05229166666666667, 199, 0.106163579135282, 0.9967261730562872), (0.013333333333333334, 227, 0.45015482737902646, 0.9943474264705883), (0.024010416666666666, 202, 0.27235002100606265, 0.9943137254901961)]
../../my_data/UCF-Brox/flow/rgb/jpg/WallPushups/v_WallPushups_g01_c01.avi/v_WallPushups_g01_c01_77.jpg
[(0.7681770833333333, 203, 0.00540541405334578, 0.9971600600373041), (0.06927083333333334, 191, 0.731853460647347, 0.9964823824266549), (0.0440625, 191, 0.5474987097164224, 0.9969359847958096), (0.05229166666666667, 199, 0.10610858132626876, 0.9967268182173268), (0.024010416666666666, 202, 0.27210895993566525, 0.9943175534856026), (0.013333333333333334, 227, 0.45015482737902646, 0.9943474264705883), (0.02885416666

In [ ]:
Execution time: 293.13425040245056 second (5)